In [1]:
import openpyxl

In [2]:
%run MyFunctions.ipynb

In [3]:
## Import data
filename = "ResearchInChildAndAd_DATA_2018-12-14_1531.xlsx"
workbook = openpyxl.load_workbook(filename)
sheet1 = workbook["ResearchInChildAndAd_DATA_2018-"]
sheet2 = workbook["Tasks"]

In [4]:
## Module 1: Make columns for new data
KEYWORDS_DICT = {'ADHD': ['attention', 'adhd', 'add'], 
                 'ASD': ['autis', 'asd'],
                 'Anxiety': ['anxiety', 'panic', 'phobi'], 
                 'Bipolar': ['bipolar', 'mani'],
                 'Depression': ['depress', 'MDD', 'PDD', 'mood', 'dysthymi'], 
                 'EOS': ['schizo'], 
                 'FASD': ['fasd', 'fetal', 'alcohol'], 
                 'OCD': ['obsess', 'compuls', 'ocd'], 
                 'ODD': ['opposit', 'defiant', 'odd'], 
                 'Parent-Child': ['parent'], 
                 'Personality': ['cluster', 'personality', 'histrio', 'borderline'], 
                 'Psychosis': ['psychosis', 'psychotic'],
                 'SUD': ['substance', 'SUD', 'use disorder'], 
                 'Trauma': ['trauma', 'adjust', 'ptsd'], 
                 'eating': ['eating', 'anorexi', 'bulimi', 'binge'], 
                 'neurodevelopmental': ['learning', 'delay', 'neurodevelopment', 'intellectual', 'conduct', 'cerebral', 'palsy', 'chromosom'],
                 'somatic': ['somati'], 
                 'suicide': ['suicid']}
# dont forget handle fads and NOT personality for EOS
# read file in instead
BINARY_HEADERS = []
for i in KEYWORDS_DICT.keys():
    BINARY_HEADERS.append(i)

## Add columns
add_column(sheet1, "main_addx")
for i in range(0, len(BINARY_HEADERS)):
    add_column(sheet1, "addx_{}".format(BINARY_HEADERS[i]))
    
add_column(sheet1, "main_dcdx")
for i in range(0, len(BINARY_HEADERS)):
    add_column(sheet1, "dcdx_{}".format(BINARY_HEADERS[i]))

In [5]:
## Create a dictionary of column names
ColNames = {}
Current  = 0
for COL in sheet1.iter_cols(1, sheet1.max_column):
    ColNames[COL[0].value] = Current
    Current += 1

In [6]:
## Set binary values to 0
for row_cells in sheet1.iter_rows(min_row=2, max_row=sheet1.max_row):
    for header in BINARY_HEADERS:
        row_cells[ColNames["addx_{}".format(header)]].value = 0
        row_cells[ColNames["dcdx_{}".format(header)]].value = 0

In [7]:
## Module 2: Fill in main diagnosis

## Admission
for row_cells in sheet1.iter_rows(min_row=2, max_row=sheet1.max_row):
    if type(row_cells[ColNames['admission_diagnosis']].value) is str:
        print("****************")
        print("record_id = {}".format(row_cells[ColNames['record_id']].value))
        
        ## Get admission diagnosis in lower-case
        full_diagnosis = row_cells[ColNames['admission_diagnosis']].value.lower()

        ## Split and iterate through statements
        diagnoses = split(full_diagnosis)
        
        if len(diagnoses) > 0:
            main_diagnosis = diagnoses[0]
            print("Main diagnosis full description: {}".format(main_diagnosis))
            
            num_main = 0
            main_illness = ""
            for illness in KEYWORDS_DICT:
                if contains(main_diagnosis, KEYWORDS_DICT[illness]):
                    num_main = num_main + 1
                    main_illness = illness
                    print("Main illness: {}".format(illness))

            row_cells[ColNames["main_addx"]].value = main_illness
            
            ## Print warning for multiple illnesses
            if num_main == 1:
                print("Non-one number of main illnesses")
            else:
                print("good")
        
        else:
            print("Length of 0 diagnosis found for {}".format(row_cells[ColNames['record_id']].value))
            



****************
record_id = 11229
Main diagnosis full description: major depressive disorder
Main illness: Depression
Non-one number of main illnesses
****************
record_id = 21235
Main diagnosis full description: major depressive disorder
Main illness: Depression
Non-one number of main illnesses
****************
record_id = 21243
Main diagnosis full description: major depressive disorder
Main illness: Depression
Non-one number of main illnesses
****************
record_id = 21277
Main diagnosis full description: major depressive disorder
Main illness: Depression
Non-one number of main illnesses
****************
record_id = 11230
Main diagnosis full description: unspecified psychosis 
Main illness: Psychosis
Non-one number of main illnesses
****************
record_id = 11231
Main diagnosis full description: generalized anxiety disorder
Main illness: Anxiety
Non-one number of main illnesses
****************
record_id = 21296
Main diagnosis full description: adjustment disorder with

Main illness: Depression
Non-one number of main illnesses
****************
record_id = 21325
Main diagnosis full description: complete school failure likely secondary to major generalized anxiety disorder and specifically
Main illness: Anxiety
Non-one number of main illnesses
****************
record_id = 21326
Main diagnosis full description: differential includes adhd
Main illness: ADHD
Non-one number of main illnesses
****************
record_id = 21327
Main diagnosis full description: unspecified depression
Main illness: Depression
Non-one number of main illnesses
****************
record_id = 21328
Main diagnosis full description: ocd
Main illness: OCD
Non-one number of main illnesses
****************
record_id = 21329
Main diagnosis full description: adjustment disorder
Main illness: Trauma
Non-one number of main illnesses
****************
record_id = 21330
Main diagnosis full description: unspecified depressive disorder
Main illness: Depression
Non-one number of main illnesses
****

Non-one number of main illnesses
****************
record_id = 21539
Main diagnosis full description: unspecified depressive disorder
Main illness: Depression
Non-one number of main illnesses
****************
record_id = 21540
Main diagnosis full description: major depressive disorder
Main illness: Depression
Non-one number of main illnesses
****************
record_id = 21541
Main diagnosis full description: asd
Main illness: ASD
Non-one number of main illnesses
****************
record_id = 21542
Main diagnosis full description: adhd
Main illness: ADHD
Non-one number of main illnesses
****************
record_id = 21543
Main diagnosis full description: autism spectrum disorder
Main illness: ASD
Non-one number of main illnesses
****************
record_id = 21545
Main diagnosis full description: mood dysregulation and trauma history
Main illness: Depression
Main illness: Trauma
good
****************
record_id = 31241
Main diagnosis full description: major depressive disorder
Main illness: 

In [8]:
IGNORE = ['query', 'vs', 'r/o']
DEPRESS = ['depress']
ADHD = ['attention']
ASD = ['autis']

In [9]:
# ## Module 3: Read Admission Diagnosis and break into substrings
# for row_cells in sheet1.iter_rows(min_row=2, max_row=sheet1.max_row):
#     if type(row_cells[ColNames['admission_diagnosis']].value) is str:
#         ## Get admission diagnosis in lower case
#         full_diagnosis = row_cells[ColNames['admission_diagnosis']].value.lower()

#         ## Split and iterate through statements
#         diagnoses = split(full_diagnosis)
#         for statement in diagnoses:

#             # Ignore certain statements
#             if not contains(statement, IGNORE):

#                 ## Check depression
#                 if contains(statement, DEPRESS):
#                     row_cells[ColNames["ADMISSION_1"]].value = 1
#                     print("DEPRESS")
#                 if contains(statement, ADHD):
#                     row_cells[ColNames["ADMISSION_2"]].value = 1  
#                     print("ADHD")
#                 if contains(statement, ASD):
#                     row_cells[ColNames["ADMISSION_3"]].value = 1
#                     print("ASD")


In [10]:
## Output file
workbook.save("output-{}.xlsx".format(getTime()))